In [ ]:
import amici
import pesto

## AMICI model

In [ ]:
import libsbml
import importlib
import os
import sys
import numpy as np

# sbml file we want to import
sbml_file = 'model_conversion_reaction.sbml'
# name of the model that will also be the name of the python module
model_name = 'model_conversion_reaction'
# directory to which the generated model code is written
model_output_dir = model_name

# import sbml model, complile and generate amici module
sbml_importer = amici.SbmlImporter(sbml_file)
sbml_importer.sbml2amici(model_name,
                         model_output_dir,
                         verbose=False)

# load amici module (the usual starting point later for the analysis)
model = model_module.getModel()
solver = model.getSolver()
solver.setSensitivityMethod(amici.AMICI_SENSI_FSA)
solver.setSensitivityOrder(amici.AMICI_SENSI_ORDER_FIRST)
edata = amici.ExpData(model.get()) # usually from file

# how to run amici now:
# rdata = amici.runAmiciSimulation(model, solver, edata)

## Optimization

In [ ]:
# create objective function from amici model
# pesto.AmiciObjective is derived from pesto.Objective, 
# the general pesto objective function class
objective = pesto.AmiciObjective(model, solver, edata, sensi_order=1)

# create optimizer object which contains all information for doing the optimization
optimizer = pesto.Optimizer()
optimizer.solver = 'bfgs|meigo'
# if select meigo -> also set default values in solver_options
optimizer.solver_options = {} # = pesto.default_options_meigo()
optimizer.startpoints = []
optimizer.startpoint_method = 'lhs|uniform|something|function'
optimizer.n_starts = 100
# see PestoOptions.m for more required options here
# returns OptimizationResult, see parameters.MS for what to return
# list of final optim results foreach multistart, times, hess, grad, 
# flags, meta information (which optimizer -> optimizer.get_repr())

# create problem object containing all information on the problem to be solved
problem = pesto.Problem(objective, parNames=None, parsToBeOptimized, lb, ub, **kwargs)
# maybe lb, ub = inf
# other constraints: kwargs, class pesto.Constraints
# constraints on pams, states, esp. pesto.AmiciConstraints (e.g. pam1 + pam2<= const)
# if optimizer cannot handle -> error
# maybe also scaling / transformation of parameters encoded here

# do the optimization
result = pesto.optimize(problem, optimizer, result=None)
# optimize is a function since it does not need an internal memory,
# just takes input and returns output in the form of a Result object
# 'result' parameter: e.g. some results from somewhere -> pick best start points

## Visualize

In [ ]:
# waterfall, parameter space, scatter plots, fits to data
# different functions for different plotting types
pesto.plot_waterfall(problem, result)

## Data storage

In [ ]:
# during optimizer run, save in sql database, maybe only after each local optimization
# (-> unique result indep. of optimizer)
# esp. for own noodles optimizers-> not only textual output but also database
# result in db when finished

## Profiles

In [ ]:
# there are three main parts: optimize, profile, sample. the overall structure of profiles and sampling
# will be similar to optimizer like above.
# we intend to only have just one result object which can be reused everywhere, but the problem of how to 
# not have one huge class but
# maybe simplified views on it for optimization, profiles and sampling is still to be solved

profiler = pesto.Profiler()

result = pesto.profile(problem, profiler, result=None)
# possibly pass result object from optimization to get good parameter guesses

## Sampling

In [ ]:
sampler = pesto.Sampler()

result = pesto.sample(problem, sampler, result=None)

In [ ]:
# open: how to parallelize. the idea is to use methods similar to those in pyabc for working on clusters.
# one way would be to specify an additional 'engine' object passed to optimize(), profile(), sample(),
# which in the default setting just does a for loop, but can also be customized.